In [1]:
%matplotlib inline
import os
import os.path
import csv
import numpy
import scipy.stats
import scipy.cluster
import itertools
import math
import sys
import sklearn.cluster
import sklearn.decomposition
import sklearn.manifold
import skimage.filters
import pandas
from matplotlib import pylab, mlab
import matplotlib.cm
import matplotlib.patches
import IPython.display
# local modules (in same dir as this notebook)
import plotutils
# import filterutils
import syspathutils


In [2]:
base_path = os.path.realpath(os.path.join(os.path.curdir, '..'))
data_path = os.path.join(base_path, 'samples')
syspathutils.append_to_sys_path('/home/gio/langdev/gitclones/arbimon2-jobs/.env/lib/python2.7/site-packages/')
syspathutils.append_to_sys_path('/home/gio/langdev/gitclones/arbimon2-jobs/lib/')

In [3]:
import a2audio.rec
import a2pyutils.storage
import a2audio.segmentation
import a2pyutils.convert

/usr/local/lib/python2.7/dist-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '


ValueError: numpy.dtype does not appear to be the correct type object

In [5]:
datasets = {x.replace('sample', '').strip('_'):os.path.join(data_path, x) for x in os.listdir(data_path)}
print '\n'.join("{} : {}".format(x, y) for x, y in datasets.items())

sabana_seca : /home/gio/langdev/gitclones/mine/fltr-audio-seg/samples/sabana_seca
el_verde : /home/gio/langdev/gitclones/mine/fltr-audio-seg/samples/el_verde
validation_data : /home/gio/langdev/gitclones/mine/fltr-audio-seg/samples/validation_data
amarakaeri : /home/gio/langdev/gitclones/mine/fltr-audio-seg/samples/amarakaeri


In [6]:
# dataset = 'smithsonian'
# dataset = 'el_verde1'
# dataset = 'sabana_seca1'
dataset = 'validation_data'
# roi_filepath = ('/home/gio/langdev/gitclones/arbimon2-jobs/synth_output/rois.txt')

roi_filepath = os.path.join(datasets[dataset], 'rois.txt')

rois_data_ori = pandas.read_csv(roi_filepath)
# compute tod and site name from recording filename
rois_data_ori['tod'] = rois_data_ori['rec'].apply(lambda x: int(x.split('.')[0].split('_')[-1].split('-')[0]) if 'undefined' not in x else 0)
# rois_data_ori['site'] = rois_data_ori['rec'].apply(lambda x: int(x.split('/')[0]) if 'undefined' not in x else 0)
rois_data_ori['site'] = rois_data_ori['rec'].apply(lambda x: 1)

print "dataset:{}, sites:{}, recs:{}, rois:{}, rois per rec:{}".format(
    dataset,
    len(rois_data_ori.groupby('site')), 
    len(rois_data_ori.groupby('rec')), 
    len(rois_data_ori), 
    len(rois_data_ori) * 1.0 / len(rois_data_ori.groupby('rec'))
)

dataset:validation_data, sites:1, recs:6, rois:2167, rois per rec:361.166666667


In [7]:
rois_data_ori

,Unnamed: 0,f0,f1,h,idx,t0,t1,w,x,y,rec,tod,site
0,0,0.000000,136.186770,1,0,0.000000,1.640405,65,0,250,Mona_DSG_1014-2013-01-15_23-55.flac,23,1
1,1,0.000000,466.926070,1,1,17.403676,17.762515,15,679,233,Mona_DSG_1014-2013-01-15_23-55.flac,23,1
2,2,0.000000,311.284047,1,2,18.659611,19.197869,22,728,241,Mona_DSG_1014-2013-01-15_23-55.flac,23,1
3,3,0.000000,252.918288,1,3,19.274763,19.556708,12,752,244,Mona_DSG_1014-2013-01-15_23-55.flac,23,1
4,4,19.455253,272.373541,1,4,13.225768,13.840920,25,516,243,Mona_DSG_1014-2013-01-15_23-55.flac,23,1
5,5,38.910506,1770.428016,1,5,2.973235,3.895963,37,116,166,Mona_DSG_1014-2013-01-15_23-55.flac,23,1
6,6,38.910506,350.194553,1,6,10.918948,11.457206,22,426,239,Mona_DSG_1014-2013-01-15_23-55.flac,23,1
7,7,77.821012,272.373541,1,7,8.125133,8.355815,10,317,243,Mona_DSG_1014-2013-01-15_23-55.flac,23,1
8,8,77.821012,505.836576,1,8,12.841298,13.174506,14,501,231,Mona_DSG_1014-2013-01-15_23-55.flac,23,1
9,9,116.731518,272.373541,1,9,10.560110,10.816423,11,412,243,Mona_DSG_1014-2013-01-15_23-55.flac,23,1


In [10]:
import collections

recs = pandas.read_csv(os.path.join(datasets[dataset], 'recs.txt'))
vals = pandas.read_csv(os.path.join(datasets[dataset], 'validations.txt'), index_col=0)

def intersects(x0, x1, y0, y1):
    return not(x1 < y0 or y1 < x0)

def linear_intersection_length(l1x1, l1x2, l2x1, l2x2):
    return max(min(l1x2, l2x2) - max(l1x1, l2x1), 0)

def rectangle_area(x1, x2, y1, y2):
    return max(x2 - x1, 0) * max(y2 - y1, 0)

def intersection_area(r1x1, r1x2, r1y1, r1y2, r2x1, r2x2, r2y1, r2y2):
    return linear_intersection_length(r1x1, r1x2, r2x1, r2x2) * linear_intersection_length(r1y1, r1y2, r2y1, r2y2)

def union_area(r1x1, r1x2, r1y1, r1y2, r2x1, r2x2, r2y1, r2y2):
    return rectangle_area(r1x1, r1x2, r1y1, r1y2) + rectangle_area(r2x1, r2x2, r2y1, r2y2) - intersection_area(r1x1, r1x2, r1y1, r1y2, r2x1, r2x2, r2y1, r2y2)


roivals, valrois = collections.defaultdict(list), collections.defaultdict(list)
for i, roi in rois_data_ori.iterrows():
    for j, val in vals.iterrows():
        area_of_union = union_area(roi.t0, roi.t1, roi.f0, roi.f1, val.t0, val.t1, val.f0, val.f1)
        area_of_intersection = intersection_area(roi.t0, roi.t1, roi.f0, roi.f1, val.t0, val.t1, val.f0, val.f1)
        val_roi_coverage = area_of_intersection / area_of_union
        if val_roi_coverage > 0:
            roivals[i].append([val_roi_coverage, j])
            valrois[j].append([val_roi_coverage, i])
            
        #if intersects(roi.t0, roi.t1, val.t0, val.t1) and intersects(roi.f0, roi.f1, val.f0, val.f1):
        #    roivals[i].append(j)
        #    valrois[j].append(i)

print len(roivals)
# len(vals) * len(rois_data_ori)
# del vals['idx']
# print vals.groupby('recording').groups.keys()
# vals.to_csv(os.path.join(datasets[dataset], 'validations.txt'))
# vals

2009


In [35]:
len(rois_data_ori) - len(roivals), len(vals) - len(valrois), len(roivals), len(valrois)

(158, 20, 2009, 2187)

In [70]:
valroi_max_overlap_percentage = [max(y[0] for y in x) for x in valrois.values()]
{
    'total':  len(valrois),
    'det_val': len(filter(lambda x: x >= .25, valroi_max_overlap_percentage))
}

{'det_val': 1879, 'total': 2187}

1661

In [ ]:
to_sample = 50
# rois_subdata_sample_full = rois_data_ori.sample(to_sample)
# rois_subdata_sample_full.to_csv(os.path.join(datasets[dataset], 'sample', 'rois.txt'))

# show_plot('bw','y_max', 'kHz', 'kHz', 6,6, X=rois_subdata)

In [ ]:
rois_subdata_sample_full = pandas.read_csv(os.path.join(datasets[dataset], 'sample', 'rois.txt'), index_col=0)
print "entries:{}, cols:{}".format(len(rois_subdata_sample_full), list(rois_subdata_sample_full.columns))

In [ ]:
def clip_pad_and_draw_rgb_bounds(X, cx0, cx1, cy0, cy1, pad, color):
    h,w = X.shape
    m, M = X.flatten().min(), X.flatten().max()
    px0, px1, py0, py1 = max(cx0-pad, 0), min(cx1+pad, w-1), max(cy0-pad, 0), min(cy1+pad, h-1)
    roi = 255 * (1 - (X[py0:py1,px0:px1] - m ) / (M - m))**.5
    rH, rW = roi.shape
    roiRGB = numpy.zeros((rH,rW,3), dtype=numpy.uint8)
    x0, x1, y0, y1= cx0-px0, cx1-px0, cy0-py0, cy1-py0
    try:
        
        x0, x1 = [max(0, min(rW-1, x)) for x in [x0,x1]]
        y0, y1 = [max(0, min(rH-1, y)) for y in [y0,y1]]
        for ch in range(3):
            roiRGB[:,:,ch] = roi
            roiRGB[y0,x0:(x1+1),ch] *= color[ch]
            roiRGB[y1,x0:(x1+1),ch] *= color[ch]
            roiRGB[(y0+1):(y1),x0,ch] *= color[ch]
            roiRGB[(y0+1):(y1),x1,ch] *= color[ch]
    except:
        print (py0,py1,px0,px1), (cy0,cy1,cx0,cx1), pad, (x0, x1, x1+1), (y0, y0+1, y1), roiRGB.shape
        raise
    
    return roiRGB

    

dataset_storage = a2pyutils.storage.from_uri(datasets[dataset])
region_rois_output = a2pyutils.storage.from_uri(os.path.join(datasets[dataset], 'sample'))
# sabana_seca1_sample_path = '/home/gio/langdev/gitclones/mine/fltr-audio-seg/samples/sabana_seca1_sample/'
# sabana_seca1_sample = os.listdir(sabana_seca1_sample_path)


# idxx = range(len(rois_subdata_sample_full))
# idxx = [0]
# idxx = [12,13,23, 40]
# idxx = [12, 13, 15, 18, 21, 23, 31, 32, 37, 40, 41, 42, 47, 48]
pylab.figure(figsize=(21,21))
# for i, (idx, roi_row) in enumerate(rois_subdata_sample_full.iloc[idxx].iterrows()):
for i, (idx, roi_row) in enumerate(rois_subdata_sample_full.iterrows()):
    print "{} of {}   idx:{} rec:{}".format(i, len(rois_subdata_sample_full), idx, roi_row['rec'])
    roi_rec = a2audio.rec.Rec(roi_row['rec'], '', dataset_storage)
    spectrum, freqs, times = roi_rec.getSpectrogram()
    specDB = a2pyutils.convert.power2decibels(spectrum)
    max_v = specDB.flatten().max()
    duration = roi_rec.samples * 1.0 / roi_rec.sample_rate
    max_freq = roi_rec.sample_rate / 2.0
    px2sec = duration / specDB.shape[1]
    px2Hz = max_freq / specDB.shape[0]
    roi_row__h__ = int((roi_row['f1'] - roi_row['f0']) / px2Hz + .5)
    
    x0, y1 = roi_row['x'], specDB.shape[0] - roi_row['y']
    x1, y0 = x0 + roi_row['w'], y1 - roi_row__h__
    
    t0, t1 = x0 * px2sec, x1 * px2sec
    f0, f1 = y0 * px2Hz, y1 * px2Hz
    
    print (x0, x1,y0, y1)
    pad=10
    roi = clip_pad_and_draw_rgb_bounds(specDB, x0, x1,y0, y1, pad, (.25,1.0,.25))
    
#     pylab.subplot(math.ceil(2*len(idxx)*1.0/2),2,2*i)
#     pylab.title("{}\n ({}-{} s)-({}-{}Hz)".format(roi_row['rec'],t0,t1,f0,f1))
#     pylab.imshow(
#          roi,
#          interpolation="nearest",
#          origin='lower'
#     )


#     pylab.subplot(math.ceil(2*len(idxx)*1.0/2),2,2*i+1)
#     spec2samp = roi_rec.original.shape[0] * 1.0 / spectrum.shape[1]
#     samp_x0, samp_x1 = int((x0-pad)*spec2samp), int((x1+pad)*spec2samp+.5)
#     roi_clip = roi_rec.original[samp_x0:samp_x1]
#     pylab.plot(numpy.log10(abs(roi_clip)) * numpy.sign(roi_clip))
    
    with region_rois_output.open_for_writing("roi-{}.png".format(i)) as fout:
        pylab.imsave(fout, 
             roi,
             origin='lower'
        )

In [ ]:
rois_data_ori['f1'] - rois_data_ori['f0']

In [ ]:
spec2samp = roi_rec.original.shape[0] * 1.0 / spectrum.shape[1]
pad=10
samp_x0, samp_x1 = int((x0-pad)*spec2samp), int((x1+pad)*spec2samp+.5)
print f0,f1
roi_clip = roi_rec.original[samp_x0:samp_x1]
pylab.figure(figsize=(15,5))

roi_clip_bpf = filterutils.butter_bandpass_filter(
    roi_rec.original, f0, f1, roi_rec.sample_rate, order=7, passes=4
)[samp_x0:samp_x1]
# roi_clip_bpf =  roi_clip

pylab.plot(roi_clip_bpf)

pylab.figure(figsize=(10,5))
pylab.subplot(1,2,1)
pylab.specgram(roi_clip_bpf)
pylab.subplot(1,2,2)
pylab.specgram(roi_clip)

IPython.display.display(IPython.display.Audio(roi_clip_bpf, rate=roi_rec.sample_rate))

In [ ]:
numpy.array([1j,2,3j,1+4j]).conjugate()

In [ ]:
for i_idx, i in enumerate(idxx):
    print "{}: ".format(i)
    roi_row1 = rois_data_ori.iloc[i]
    (roi1, roi1_bounds), _1, _2 = get_roi(roi_row1, sabana_seca1_sample_storage)
    with region_rois_output.open_for_writing("data-roi-{}.png".format(i)) as fout:
        numpy.save(fout, roi1)


In [ ]:
def get_roi(RR, storage, pad=10):
    roi_rec = a2audio.rec.Rec(RR['rec'], '', storage)
    spectrum, freqs, times = roi_rec.getSpectrogram()
    specDB = a2pyutils.convert.power2decibels(spectrum)
    max_v = specDB.flatten().max()
    duration = roi_rec.samples * 1.0 / roi_rec.sample_rate
    max_freq = roi_rec.sample_rate / 2.0
    px2sec = duration / specDB.shape[1]
    px2Hz = max_freq / specDB.shape[0]

    x0, y1 = RR['x'], specDB.shape[0] - RR['y']
    x1, y0 = x0 + RR['w'], y1 - RR['h']
    
    return crop(specDB, x0, x1, y0, y1, pad), (duration, max_freq, max_v), (px2sec, px2Hz)

def crop(X, x0, x1, y0, y1, pad=10):
    h,w = X.shape
    px0, px1, py0, py1 = max(x0-pad, 0), min(x1+pad, w-1), max(y0-pad, 0), min(y1+pad, h-1)
    roi = X[py0:py1,px0:px1].astype(numpy.float32)
    return roi, (px0, py0, px1, py1)

def correl(X, Y):
    (xh,xw), (yh,yw) = X.shape, Y.shape
    pX = numpy.pad(X, ((0,yh),(0,yw)), mode='constant', constant_values=(float(numpy.median(X)),))
    pY = numpy.pad(Y, ((xh,0),(xw,0)), mode='constant', constant_values=(float(numpy.median(Y)),))
    fX, fY = numpy.fft.fft2(pX), numpy.fft.fft2(pY)
    fCorr = (fX * fY.conjugate())
    fCorr /= abs(fX) * abs(fY)
    Corr = numpy.fft.ifft2(fCorr)
    return abs(Corr)

def roi_dist(roi1, roi2):
    C = correl(roi1, roi2)
#     pylab.imshow(C, interpolation="nearest")
    return C.max()


sabana_seca1_sample_storage = a2pyutils.storage.from_uri('/home/gio/langdev/gitclones/mine/fltr-audio-seg/samples/sabana_seca1_sample/')
region_rois_output = a2pyutils.storage.from_uri('/home/gio/langdev/gitclones/mine/fltr-audio-seg/samples/sabana_seca1_sample/region_bw_700_1900_ymax_2650_3550')
# sabana_seca1_sample_path = '/home/gio/langdev/gitclones/mine/fltr-audio-seg/samples/sabana_seca1_sample/'
# sabana_seca1_sample = os.listdir(sabana_seca1_sample_path)


print "total number of rois : {}".format(len(rois_data_ori))

idxx = range(len(rois_data_ori))
# idxx = [12,13,23, 40]
# idxx = [12, 13, 15, 18, 21, 23, 31, 32, 37, 40, 41, 42, 47, 48]
# idxx = [0, 1, 2, 3, 4, 5]
# pylab.figure(figsize=(21,21))

with region_rois_output.open_for_writing("roi-dist.txt") as fout:
    fout.write("i, j, d\n")
    for i_idx, i in enumerate(idxx):
        print "{}: ".format(i)
        roi_row1 = rois_data_ori.iloc[i]
        (roi1, roi1_bounds), _1, _2 = get_roi(roi_row1, sabana_seca1_sample_storage)
        for j_idx, j in enumerate(idxx[i_idx+1:]):
            roi_row2 = rois_data_ori.iloc[j]
            (roi2, roi2_bounds), _1, _2 = get_roi(roi_row2, sabana_seca1_sample_storage)
            d = roi_dist(roi1, roi2)
            print "   vs {} : sim : {}".format(j, d)
            fout.write("{}, {}, {}\n".format(i, j, d))
# #     with region_rois_output.open_for_writing("roi2-{}.png".format(i)) as fout:
# #         pylab.imsave(fout, 
# #              roi,
# #              origin='lower'
# #         )

In [ ]:
(roi1, roi1_bounds), _1, _2 = get_roi(rois_data_ori.iloc[4], sabana_seca1_sample_storage)
(roi2, roi2_bounds), _1, _2 = get_roi(rois_data_ori.iloc[5], sabana_seca1_sample_storage)
(roi3, roi2_bounds), _1, _2 = get_roi(rois_data_ori.iloc[3], sabana_seca1_sample_storage)
# d = roi_dist(roi1, roi2)
pylab.subplot(1,3,1)
pylab.imshow(roi1)
pylab.subplot(1,3,2)
pylab.imshow(roi2)
pylab.subplot(1,3,3)
pylab.imshow(roi3)
